In [ ]:
!pip install langchain faiss-cpu pandas openai groq python-dotenv
!pip install spacy transformers
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 118.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
import spacy
from transformers import pipeline

# Load spaCy model for NER
nlp = spacy.load("en_core_web_sm")

# Load FinBERT sentiment model
sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
GROQ_API_KEY='yourkey'

In [ ]:
def annotate_news(text):
    doc = nlp(text)
    entities = list(set([ent.text.strip() for ent in doc.ents if ent.label_ in ("ORG", "PERSON")]))
    sentiment_result = sentiment_pipeline(text[:512])[0]  # truncate to 512 tokens
    sentiment = sentiment_result["label"]
    return {
        "entities": entities,
        "sentiment": sentiment
    }


In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import spacy
from transformers import pipeline

# Load spaCy model and FinBERT
nlp = spacy.load("en_core_web_sm")
sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# Load scraped CSV
df = pd.read_csv("/content/moneycontrol_financial_news.csv")

# Step 1: Split into chunks and collect texts + base metadata
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

chunk_texts = []
chunk_metadatas = []

for _, row in df.iterrows():
    content = row['full_article']
    base_metadata = {
        "company": row['company'],
        "year": row['year'],
        "title": row['title'],
        "link": row['link']
    }
    chunks = splitter.split_text(content)
    for chunk in chunks:
        chunk_texts.append(chunk)
        chunk_metadatas.append(base_metadata)

# Step 2: Batch sentiment analysis (you can tune batch_size as needed)
sentiment_results = sentiment_pipeline(chunk_texts, batch_size=16, truncation=True)

# Step 3: Apply spaCy NER and combine annotations
texts = []
metadatas = []

for chunk, sentiment_result, base_metadata in zip(chunk_texts, sentiment_results, chunk_metadatas):
    doc = nlp(chunk)
    entities = list(set([ent.text.strip() for ent in doc.ents if ent.label_ in ("ORG", "PERSON")]))
    sentiment = sentiment_result["label"]
    annotation = {"entities": entities, "sentiment": sentiment}
    combined_metadata = {**base_metadata, **annotation}
    texts.append(chunk)
    metadatas.append(combined_metadata)


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Wrap texts and metadata into Document objects
documents = [Document(page_content=text, metadata=meta) for text, meta in zip(texts, metadatas)]

# Build FAISS vectorstore
vectorstore = FAISS.from_documents(documents, embedding_model)

# Save locally
vectorstore.save_local("moneycontrol_faiss_index")


/tmp/ipython-input-961244428.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
!pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

# Set API Key
import os
os.environ["GROQ_API_KEY"] = 'Yourkey'

llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model_name="meta-llama/llama-4-scout-17b-16e-instruct",  # Or llama3-70b-8192, gemma-7b
)

# Build RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # or "refine" or "map_reduce" depending on use case
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True
)


In [ ]:
# query = "What news came out about TCS?"
# result = qa(query)

# print("Answer:", result['result'])

# print("\nSources:")
# for doc in result['source_documents']:
#     print("-", doc.metadata['title'], " | ", doc.metadata['link'])


In [ ]:
# retriever = vectorstore.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 10, "score_threshold": 0.7}
# )


In [ ]:
company_map = {
    'hdfc': {'code': 'HDF01', 'ticker': 'HDFCBANK.NS'},
    'icici': {'code': 'ICI02', 'ticker': 'ICICIBANK.NS'},
    'state bank of india(sbi)': {'code': 'SBI', 'ticker': 'SBIN.NS'},
    'kotak mahindra': {'code': 'KMB', 'ticker': 'KOTAKBANK.NS'},
    'axis': {'code': 'AB16', 'ticker': 'AXISBANK.NS'},
    'bank of baroda': {'code': 'BOB', 'ticker': 'BANKBARODA.NS'},
    'punjab national': {'code': 'PNB05', 'ticker': 'PNB.NS'},
    'union bank of india': {'code': 'UBI01', 'ticker': 'UNIONBANK.NS'},
    'canara bank': {'code': 'CB06', 'ticker': 'CANBK.NS'},
    'indusind': {'code': 'IIB', 'ticker': 'INDUSINDBK.NS'},
    'bajaj finance': {'code': 'BAF', 'ticker': 'BAJFINANCE.NS'},
    'muthoot finance': {'code': 'MF10', 'ticker': 'MUTHOOTFIN.NS'},
    'shriram finance': {'code': 'STF', 'ticker': 'SHRIRAMFIN.NS'},
    'cholamandalam invest.': {'code': 'CDB', 'ticker': 'CHOLAFIN.NS'},
    'l&t finance holdings': {'code': 'LFH', 'ticker': 'L&TFH.NS'},
    'aditya birla finance': {'code': 'ABC9', 'ticker': 'ABFRL.NS'},
    'mahindra finance': {'code': 'MMF04', 'ticker': 'M&MFIN.NS'},
    'poonawalla fincorp': {'code': 'MF20', 'ticker': 'POONAWALLA.NS'},
    'reliance': {'code': 'RI', 'ticker': 'RELIANCE.NS'},
    'tata consultancy services (tcs)':{'code': 'TCS', 'ticker': 'TCS.NS'},
    'tcs': {'code': 'TCS', 'ticker': 'TCS.NS'},
    'infosys': {'code': 'IT', 'ticker': 'INFY.NS'},
    'airtel': {'code': 'BA08', 'ticker': 'BHARTIARTL.NS'},
    'hindustan unilever': {'code': 'HU', 'ticker': 'HINDUNILVR.NS'},
    'lic': {'code': 'LIC09', 'ticker': 'LICI.NS'}
}


In [ ]:
def detect_company_in_query(query):
    query_lower = query.lower()
    for name, mapping in company_map.items():
        if name in query_lower:
            return name, mapping['code'], mapping['ticker']
    return None, None, None

In [ ]:
user_query = "What are the sentiments around indusind this month?"

# Detect company name, code, ticker from query
company_name, company_code, ticker = detect_company_in_query(user_query)  # use user_query here

if company_code:
    # retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5, "filter": {"company": company_code}}
    )
else:
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
You are a financial news assistant. Use the following context to answer:

{context}

Question: {question}
Answer:
"""
)

# Make sure to pass the prompt if you want to customize prompt usage
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

result = qa(user_query)

print("Answer:", result['result'])

print("\nUnique Sources:")
seen_links = set()
for doc in result['source_documents']:
    metadata = doc.metadata
    key = metadata.get('link') or metadata.get('title')
    if key and key not in seen_links:
        seen_links.add(key)
        if 'title' in metadata and 'link' in metadata:
            print("-", metadata['title'], "|", metadata['link'])
        else:
            print("- Source document metadata:", metadata)
            print("  Content preview:", doc.page_content[:200] + "...")

/tmp/ipython-input-1289674896.py:36: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(user_query)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Answer: The sentiments around IndusInd Bank this month appear to be negative, with analysts expressing concerns and a "crisis of confidence" in the bank. Multiple brokerages have slashed their earnings estimates and downgraded the stock following the bank's March quarter results. Although the bank's management has assured that all known discrepancies have been accounted for and a stricter compliance and governance culture is being instilled, analysts and the market remain wary. The discovery of fresh irregularities, such as misrecognised fee income, just a day before the results, has also raised doubts about whether all issues have been fully addressed.

Unique Sources:
- Crisis of confidence at IndusInd Bank, say analysts, lender assures clean-up is underway | https://www.moneycontrol.com/banking/crisis-of-confidence-at-indusind-bank-say-analysts-lender-assures-clean-up-is-underway-article-13040947.html
- Can IndusInd bury its troubled past and move on? | https://www.moneycontrol.com/

In [ ]:
if company_name:
    print("Latest updates for", company_name.title() + ":")
else:
    print("No company detected in the query.")

print(result['result'])

print("\nSources:")
for doc in result['source_documents']:
    meta = doc.metadata
    print("-", meta.get('title', ''), "|", meta.get('link', ''))

Latest updates for Indusind:
The sentiments around IndusInd Bank this month appear to be negative, with analysts expressing concerns and a "crisis of confidence" in the bank. Multiple brokerages have slashed their earnings estimates and downgraded the stock following the bank's March quarter results. Although the bank's management has assured that all known discrepancies have been accounted for and a stricter compliance and governance culture is being instilled, analysts and the market remain wary. The discovery of fresh irregularities, such as misrecognised fee income, just a day before the results, has also raised doubts about whether all issues have been fully addressed.

Sources:
- Crisis of confidence at IndusInd Bank, say analysts, lender assures clean-up is underway | https://www.moneycontrol.com/banking/crisis-of-confidence-at-indusind-bank-say-analysts-lender-assures-clean-up-is-underway-article-13040947.html
- Crisis of confidence at IndusInd Bank, say analysts, lender assure

In [ ]:
import yfinance as yf
import datetime as dt
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()  # Displays Bokeh plots inline in Jupyter

def plot_stock_chart(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date)
    if df.empty:
        print("No data found for this ticker.")
        return
    p = figure(title=f"Stock Price: {ticker}", x_axis_type="datetime", width=800, height=400)
    p.line(df.index, df['Close'], color='blue', legend_label="Close Price")
    p.legend.location = "top_left"
    p.xaxis.axis_label = "Date"
    p.yaxis.axis_label = "Price"
    show(p)

# Example usage: set ticker as detected (e.g., "POONAWALLA.NS")
start_date = "2024-01-01"
end_date = dt.datetime.now().strftime("%Y-%m-%d")
plot_stock_chart(ticker, start_date, end_date)

/tmp/ipython-input-2042116649.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [ ]:
!pip install streamlit pyngrok

In [ ]:
!pip install streamlit-bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.0 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
import threading
from google.colab import userdata

# Kill previous tunnels if any
ngrok.kill()

# Get the ngrok authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open a public tunnel to the Streamlit port (8501)
# Change port to 8501 as Streamlit runs on 8501 by default
public_url = ngrok.connect(8501)
print("Streamlit app running at:", public_url)

# Run Streamlit in a background thread
def run_streamlit():
    !streamlit run app.py --server.enableCORS false --server.enableXsrfProtection false

thread = threading.Thread(target=run_streamlit)
thread.start()

In [ ]:
%%writefile app.py
import os
import streamlit as st
import pandas as pd
import datetime as dt
import yfinance as yf
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.embed import components
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq

# Load FAISS vectorstore
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("moneycontrol_faiss_index", embedding_model, allow_dangerous_deserialization=True)

# Setup LLM
os.environ["GROQ_API_KEY"] = "yourkey"
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="meta-llama/llama-4-scout-17b-16e-instruct")

# Define prompt
prompt = PromptTemplate.from_template(
    """
You are a helpful financial assistant. Use the context to answer the user's question accurately.
Context:
{context}

Question: {question}
Answer in 2–3 short paragraphs:
"""
)

# Company lookup
company_map = {
    'hdfc': {'code': 'HDF01', 'ticker': 'HDFCBANK.NS'},
    'icici': {'code': 'ICI02', 'ticker': 'ICICIBANK.NS'},
    'state bank of india(sbi)': {'code': 'SBI', 'ticker': 'SBIN.NS'},
    'kotak mahindra': {'code': 'KMB', 'ticker': 'KOTAKBANK.NS'},
    'axis': {'code': 'AB16', 'ticker': 'AXISBANK.NS'},
    'bank of baroda': {'code': 'BOB', 'ticker': 'BANKBARODA.NS'},
    'punjab national': {'code': 'PNB05', 'ticker': 'PNB.NS'},
    'union bank of india': {'code': 'UBI01', 'ticker': 'UNIONBANK.NS'},
    'canara bank': {'code': 'CB06', 'ticker': 'CANBK.NS'},
    'indusind': {'code': 'IIB', 'ticker': 'INDUSINDBK.NS'},
    'bajaj finance': {'code': 'BAF', 'ticker': 'BAJFINANCE.NS'},
    'muthoot finance': {'code': 'MF10', 'ticker': 'MUTHOOTFIN.NS'},
    'shriram finance': {'code': 'STF', 'ticker': 'SHRIRAMFIN.NS'},
    'cholamandalam invest.': {'code': 'CDB', 'ticker': 'CHOLAFIN.NS'},
    'l&t finance holdings': {'code': 'LFH', 'ticker': 'L&TFH.NS'},
    'aditya birla finance': {'code': 'ABC9', 'ticker': 'ABFRL.NS'},
    'mahindra finance': {'code': 'MMF04', 'ticker': 'M&MFIN.NS'},
    'poonawalla fincorp': {'code': 'MF20', 'ticker': 'POONAWALLA.NS'},
    'reliance': {'code': 'RI', 'ticker': 'RELIANCE.NS'},
    'tata consultancy services (tcs)':{'code': 'TCS', 'ticker': 'TCS.NS'},
    'tcs': {'code': 'TCS', 'ticker': 'TCS.NS'},
    'infosys': {'code': 'IT', 'ticker': 'INFY.NS'},
    'airtel': {'code': 'BA08', 'ticker': 'BHARTIARTL.NS'},
    'hindustan unilever': {'code': 'HU', 'ticker': 'HINDUNILVR.NS'},
    'lic india': {'code': 'LIC09', 'ticker': 'LICI.NS'} # Corrected company name
}

def detect_company(query):
    query_lower = query.lower()
    for name, mapping in company_map.items():
        if name in query_lower:
            return name, mapping['code'], mapping['ticker']
    return None, None, None

def get_news_response(query, retriever):
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        return_source_documents=True
    )
    return qa(query)

import numpy as np
import math
import datetime as dt
from bokeh.plotting import figure
from streamlit_bokeh import streamlit_bokeh

def plot_stock_chart(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date)
    if df.empty:
        return None

    p = figure(title=f"{ticker} Stock Price", x_axis_type="datetime", width=800, height=400)
    p.line(df.index, df['Close'], color='blue', legend_label="Close Price")
    p.add_tools(HoverTool(
        tooltips=[("Date", "@x{%F}"), ("Close", "@y")],
        formatters={'@x': 'datetime'},
        mode='vline'
    ))
    p.xaxis.axis_label = "Date"
    p.yaxis.axis_label = "Price"
    p.legend.location = "top_left"

    return p  # return Bokeh figure



# --- Streamlit UI ---
st.set_page_config(page_title="Financial News Assistant", layout="wide")
st.title("📊 Financial News Assistant")

user_query = st.text_input("Ask a question about any company (e.g., 'Updates on HDFC Bank'):")

if user_query:
    company_name, company_code, ticker = detect_company(user_query)

    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5}
        # Optional: enable filter like this
        # search_kwargs={"k": 5, "filter": {"company": company_code}} if company_code else ...
    )

    result = get_news_response(user_query, retriever)
    from transformers import pipeline
    sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")
    sentiment = sentiment_pipeline(user_query)[0]

    st.subheader("💬 Sentiment Analysis")
    st.write(f"**Label:** {sentiment['label']} | **Confidence:** {sentiment['score']:.2f}")


    st.subheader("📌 Answer")
    st.write(result["result"])

    st.subheader("🔎 Source Articles")
    seen_links = set()
    for doc in result["source_documents"]:
        meta = doc.metadata
        key = meta.get("link") or meta.get("title")
        if key and key not in seen_links:
            seen_links.add(key)
            title = meta.get("title", "Untitled Article")
            link = meta.get("link", "#")
            st.markdown(f"- [{title}]({link})")

    # Show chart if ticker found
    if ticker:
        st.subheader("📈 Stock Price Trend")
        start_date = "2024-01-01"
        end_date = dt.datetime.today().strftime("%Y-%m-%d")
        chart = plot_stock_chart(ticker, start_date, end_date)
        if chart:
            from streamlit_bokeh import streamlit_bokeh
            streamlit_bokeh(chart, use_container_width=True, theme="streamlit", key="stock_chart")
        else:
            st.warning("Stock price data not available.")


Overwriting app.py


# in langgraph

In [ ]:
!pip install langgraph

In [ ]:
%%writefile app.py
import os
import datetime as dt
import yfinance as yf
import streamlit as st
from bokeh.plotting import figure
from bokeh.models import HoverTool
from streamlit_bokeh import streamlit_bokeh

# New
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from transformers import pipeline

from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional

# --- Setup FAISS vectorstore ---
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("moneycontrol_faiss_index", embedding_model, allow_dangerous_deserialization=True)

# --- Setup LLM ---
os.environ["GROQ_API_KEY"] = 'yourkey'
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="meta-llama/llama-4-scout-17b-16e-instruct")

# --- Prompt ---
prompt = PromptTemplate.from_template("""
You are a helpful financial assistant. Use the context to answer the user's question accurately.
Context:
{context}

Question: {question}
Answer in 2–3 short paragraphs:
""")

# --- Company map ---
company_map = {
    'hdfc': {'code': 'HDF01', 'ticker': 'HDFCBANK.NS'},
    'icici': {'code': 'ICI02', 'ticker': 'ICICIBANK.NS'},
    'state bank of india(sbi)': {'code': 'SBI', 'ticker': 'SBIN.NS'},
    'kotak mahindra': {'code': 'KMB', 'ticker': 'KOTAKBANK.NS'},
    'axis': {'code': 'AB16', 'ticker': 'AXISBANK.NS'},
    'bank of baroda': {'code': 'BOB', 'ticker': 'BANKBARODA.NS'},
    'punjab national': {'code': 'PNB05', 'ticker': 'PNB.NS'},
    'union bank of india': {'code': 'UBI01', 'ticker': 'UNIONBANK.NS'},
    'canara bank': {'code': 'CB06', 'ticker': 'CANBK.NS'},
    'indusind': {'code': 'IIB', 'ticker': 'INDUSINDBK.NS'},
    'bajaj finance': {'code': 'BAF', 'ticker': 'BAJFINANCE.NS'},
    'muthoot finance': {'code': 'MF10', 'ticker': 'MUTHOOTFIN.NS'},
    'shriram finance': {'code': 'STF', 'ticker': 'SHRIRAMFIN.NS'},
    'cholamandalam invest.': {'code': 'CDB', 'ticker': 'CHOLAFIN.NS'},
    'l&t finance holdings': {'code': 'LFH', 'ticker': 'L&TFH.NS'},
    'aditya birla finance': {'code': 'ABC9', 'ticker': 'ABFRL.NS'},
    'mahindra finance': {'code': 'MMF04', 'ticker': 'M&MFIN.NS'},
    'poonawalla fincorp': {'code': 'MF20', 'ticker': 'POONAWALLA.NS'},
    'reliance': {'code': 'RI', 'ticker': 'RELIANCE.NS'},
    'tata consultancy services (tcs)': {'code': 'TCS', 'ticker': 'TCS.NS'},
    'tcs': {'code': 'TCS', 'ticker': 'TCS.NS'},
    'infosys': {'code': 'IT', 'ticker': 'INFY.NS'},
    'airtel': {'code': 'BA08', 'ticker': 'BHARTIARTL.NS'},
    'hindustan unilever': {'code': 'HU', 'ticker': 'HINDUNILVR.NS'},
    'lic india': {'code': 'LIC09', 'ticker': 'LICI.NS'}
}

def detect_company(query):
    query_lower = query.lower()
    for name, mapping in company_map.items():
        if name in query_lower:
            return name, mapping['code'], mapping['ticker']
    return None, None, None

# --- Sentiment pipeline ---
sentiment_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# --- Plotting function ---
def plot_stock_chart(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date)
    if df.empty:
        return None
    p = figure(title=f"{ticker} Stock Price", x_axis_type="datetime", width=800, height=400)
    p.line(df.index, df['Close'], color='blue', legend_label="Close Price")
    p.add_tools(HoverTool(
        tooltips=[("Date", "@x{%F}"), ("Close", "@y")],
        formatters={'@x': 'datetime'},
        mode='vline'
    ))
    p.xaxis.axis_label = "Date"
    p.yaxis.axis_label = "Price"
    p.legend.location = "top_left"
    return p

# --- LangGraph State ---
class GraphState(TypedDict):
    query: str
    company_name: Optional[str]
    company_code: Optional[str]
    ticker: Optional[str]
    news_result: Optional[dict]
    sentiment: Optional[dict]

# --- Graph nodes ---
def detect_company_node(state: GraphState):
    name, code, ticker = detect_company(state["query"])
    return {**state, "company_name": name, "company_code": code, "ticker": ticker}

def retrieve_news_node(state: GraphState):
    if state.get("company_code"):
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5, "filter": {"company": state["company_code"]}})
    else:
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff", return_source_documents=True)
    res = qa(state["query"])
    return {**state, "news_result": res}

def analyze_sentiment_node(state: GraphState):
    sentiment = sentiment_pipeline(state["query"])[0]
    return {**state, "sentiment": sentiment}

# --- Graph builder ---
graph = StateGraph(GraphState)
graph.add_node("detect_company", detect_company_node)
graph.add_node("retrieve_news", retrieve_news_node)
graph.add_node("analyze_sentiment", analyze_sentiment_node)
graph.add_edge("detect_company", "retrieve_news")
graph.add_edge("retrieve_news", "analyze_sentiment")
graph.set_entry_point("detect_company")
graph.add_edge("analyze_sentiment", END)
app_graph = graph.compile()

# --- Streamlit UI ---
st.set_page_config(page_title="Financial News Assistant", layout="wide")
st.title("Financial News Assistant")

user_query = st.text_input("Ask a question about any company:")

if user_query:
    final_state = app_graph.invoke({"query": user_query})

    st.subheader("Sentiment Analysis")
    st.write(f"Label: {final_state['sentiment']['label']} | Confidence: {final_state['sentiment']['score']:.2f}")

    st.subheader("Answer")
    st.write(final_state["news_result"]["result"])

    st.subheader("Source Articles")
    seen_links = set()
    for doc in final_state["news_result"]["source_documents"]:
        meta = doc.metadata
        key = meta.get("link") or meta.get("title")
        if key and key not in seen_links:
            seen_links.add(key)
            st.markdown(f"- [{meta.get('title', 'Untitled Article')}]({meta.get('link', '#')})")

    if final_state.get("ticker"):
        st.subheader("Stock Price Trend")
        start_date = "2024-01-01"
        end_date = dt.datetime.today().strftime("%Y-%m-%d")
        chart = plot_stock_chart(final_state["ticker"], start_date, end_date)
        if chart:
            streamlit_bokeh(chart, use_container_width=True, theme="streamlit", key="stock_chart")
        else:
            st.warning("Stock price data not available.")


Overwriting app.py


In [ ]:
from pyngrok import ngrok
import threading
import time
import os

# Kill previous tunnels if any
ngrok.kill()

# Start ngrok tunnel with correct syntax (HTTP scheme)
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app will be available at: {public_url}")

# Run Streamlit app in a thread
def run():
    os.system("streamlit run app.py --server.enableCORS false")

thread = threading.Thread(target=run)
thread.start()


Streamlit app will be available at: NgrokTunnel: "https://3539922ad113.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, RetrievalQA
import os

# --- Load FAISS index ---
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("moneycontrol_faiss_index", embedding_model, allow_dangerous_deserialization=True)

# --- LLaMA-4 setup ---
os.environ["GROQ_API_KEY"] = "yourkey"
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model_name="meta-llama/llama-4-scout-17b-16e-instruct")

# --- Prompt for percentage-based fact-checking ---
faithfulness_prompt = PromptTemplate.from_template("""
You are a fact-checker. Compare the ANSWER with the CONTEXT.
Estimate the percentage of claims in the ANSWER that are directly supported by the CONTEXT.
Return only a number between 0 and 100.

CONTEXT:
{context}

ANSWER:
{answer}
""")
faithfulness_chain = LLMChain(llm=llm, prompt=faithfulness_prompt)

# --- QA Chain to get LLM answer using retrieved docs ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    chain_type="stuff",
    return_source_documents=True
)

# --- Test queries ---
test_queries = [
    "Sentiment around HDFC?",
    "What did Infosys say about its revenue forecast?",
]

scores = []

for query in test_queries:
    # Get LLM answer + retrieved docs
    res = qa_chain(query)
    answer = res["result"]
    docs = res["source_documents"]
    context = "\n\n".join([doc.page_content for doc in docs])

    # Calculate faithfulness score
    score_str = faithfulness_chain.run({"context": context, "answer": answer}).strip()
    try:
        score = float(score_str)
    except ValueError:
        score = 0.0
    scores.append(score)

    # Show results
    print(f"Query: {query}")
    print(f"LLM Answer: {answer}")
    print(f"Faithfulness Score: {score:.2f}%")
    print("\n--- Retrieved Sources ---")
    for i, doc in enumerate(docs, start=1):
        print(f"[{i}] {doc.metadata.get('title', 'Untitled')} - {doc.metadata.get('link', 'No link')}")
        print(doc.page_content[:300] + "...\n")
    print("=" * 100)

# --- Final metric ---
avg_faithfulness = sum(scores) / len(scores)
print(f"Average Faithfulness Score: {avg_faithfulness:.2f}%")


Query: Sentiment around HDFC?
LLM Answer: The sentiment around HDFC Bank appears to be positive. Jefferies, an international brokerage, has reiterated its bullishness on HDFC Bank, retaining it as one of its top picks in the banking sector. They have also increased their target price for the stock to Rs 2,270. Additionally, HDFC Bank's shares have gained over 20% in the past year and over 1% on the day of the report, indicating a positive market sentiment. Other brokerages, such as Emkay Global, also seem to be optimistic about the bank's prospects, citing its resilient balance sheet and potential for growth.
Faithfulness Score: 0.00%

--- Retrieved Sources ---
[1] Accumulate HDFC Bank; target of Rs 2274: Deven Choksey - https://www.moneycontrol.com/news/business/stocks/accumulate-hdfc-bank-target-of-rs-2274-deven-choksey-13325575.html
certified experts before taking any investment decisions. HDFC BANK - 25072025 - deven Discover the latestBusiness News,Budget 2025 News,Sensex, andNift